## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Sampler

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

## Set up the Qiskit runtime session

In [4]:
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='system-request/7-8-22-access/main',
    token="af0c18b11e3ac4f44582dcccc09b543c998e3caacb5bc151f059ef098195be53f1e33c5c71933bb3c1e68fc8173fc67e5433f6a2a0d7457ee875b6712c2cee1a",
)
session = Session(service=service, backend='ibmq_qasm_simulator')

### Cut the circuit and evaluate the subcircuits within a Qiskit Session context

In [5]:
options = Options(resilience_level=1, optimization_level=3, execution={'shots': 8192})
with Sampler(session=session, options=options) as sampler:
    cutter = WireCutter(qc, sampler)
    cuts = cutter.cut_manual(subcircuit_vertices=[[0, 1], [2, 3]])
    subcircuit_instance_probabilities = cutter.evaluate(cuts)

--------------------
subcircuit 0
ρ qubits = 0, O qubits = 1, width = 3, effective = 2, depth = 6, size = 12
     ┌───┐                     ┌─────────┐┌───┐┌───┐
q_0: ┤ H ├──■───────────────■──┤ Rx(π/2) ├┤ T ├┤ H ├
     ├───┤┌─┴─┐┌─────────┐  │  └──┬───┬──┘├───┤└───┘
q_1: ┤ H ├┤ X ├┤ Rx(π/2) ├──┼─────┤ T ├───┤ H ├─────
     ├───┤├───┤└─────────┘┌─┴─┐   └───┘   └───┘     
q_2: ┤ H ├┤ T ├───────────┤ X ├─────────────────────
     └───┘└───┘           └───┘                     
subcircuit 1
ρ qubits = 1, O qubits = 0, width = 3, effective = 3, depth = 6, size = 11
                                          ┌───┐
q_0: ───────────────────────■───────■─────┤ H ├
     ┌───┐┌───┐             │     ┌─┴─┐   ├───┤
q_1: ┤ H ├┤ T ├─────────────┼─────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐┌─┴─┐┌──┴───┴──┐├───┤
q_2: ┤ H ├┤ T ├┤ Rx(π/2) ├┤ X ├┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘└───┘└─────────┘└───┘
Estimated cost = 1.280e+02


/Users/caleb/opt/anaconda3/envs/ckt/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The BaseSampler.__enter__ method is deprecated as of Qiskit Terra 0.22.0 and will be removed no sooner than 3 months after the releasedate. BaseSampler should be initialized directly.
  


--------------------


/Users/caleb/opt/anaconda3/envs/ckt/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The BaseSampler.__exit__ method is deprecated as of Qiskit Terra 0.22.0 and will be removed no sooner than 3 months after the releasedate. BaseSampler should be initialized directly.
  """


## Recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [6]:
ordered_probabilities = cutter.recompose(subcircuit_instance_probabilities, cuts, num_threads=4)
metrics = cutter.verify(ordered_probabilities)
print(metrics)

{'nearest': {'chi2': 0.0005373468790461289, 'Mean Squared Error': 9.976223771626921e-07, 'Mean Absolute Percentage Error': 4.474402769622725, 'Cross Entropy': 2.600218191786652, 'HOP': 0.8999142046429887}, 'naive': {'chi2': 0.0005373468790461297, 'Mean Squared Error': 9.976223771626944e-07, 'Mean Absolute Percentage Error': 4.474402769622727, 'Cross Entropy': 2.6002181917866514, 'HOP': 0.8999142046429888}}
